In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import json

In [2]:
# sample_data load
with open('sample_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
# # "usage_example" 값을 추출
# usage_examples = [item['FoundedEntity']['usage_example'] for item in data]

# "usage_example" 첫번째 key의 값을 추출
usage_examples = data[0]['FoundedEntity']['usage_example']

In [4]:
usage_examples

"It was a bit challenging, especially the past perfect tense, but I think I'm getting the hang of it."

In [5]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('/Users/marketdesigners/Documents/GitHub/sentencec-bert/model')
model = AutoModel.from_pretrained('/Users/marketdesigners/Documents/GitHub/sentencec-bert/model')

In [6]:
# Tokenize sentences
encoded_input = tokenizer(usage_examples, padding=True, truncation=True, return_tensors='pt')

In [7]:
# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

: 

In [2]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)